In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep

In [20]:
pages = []
for number in range(1, 61):
    URL = f"https://www.politifact.com/punditfact/statements/?page={number}" 
    print("Scraping", URL)
    response = requests.get(URL) 
    soup = BeautifulSoup(response.content) 
    rows = soup.find_all('div', class_='scoretable__item')
    for row in rows:
        row_dict = {}
        row_dict['Score'] = (row.find('div', class_="meter").a.img.get('alt'))
        row_dict['Date'] = (row.find('span', class_="article__meta").text)
        row_dict['In reality'] = (row.find('div', class_="meter").find('p', class_="quote").text)
        row_dict['Source'] = (row.find('div', class_="statement__source").text.strip())
        row_dict['Statement'] = (row.find('p', class_="statement__text").text.strip())
        row_dict['Link'] = (row.find('p', class_="statement__text").a.get('href'))
        row_dict['PolitiFact source'] = (row.find('p', class_="statement__edition").a.text.strip())
        pages.append(row_dict)

Scraping https://www.politifact.com/punditfact/statements/?page=1
Scraping https://www.politifact.com/punditfact/statements/?page=2
Scraping https://www.politifact.com/punditfact/statements/?page=3
Scraping https://www.politifact.com/punditfact/statements/?page=4
Scraping https://www.politifact.com/punditfact/statements/?page=5
Scraping https://www.politifact.com/punditfact/statements/?page=6
Scraping https://www.politifact.com/punditfact/statements/?page=7
Scraping https://www.politifact.com/punditfact/statements/?page=8
Scraping https://www.politifact.com/punditfact/statements/?page=9
Scraping https://www.politifact.com/punditfact/statements/?page=10
Scraping https://www.politifact.com/punditfact/statements/?page=11
Scraping https://www.politifact.com/punditfact/statements/?page=12
Scraping https://www.politifact.com/punditfact/statements/?page=13
Scraping https://www.politifact.com/punditfact/statements/?page=14
Scraping https://www.politifact.com/punditfact/statements/?page=15
Scra

In [21]:
df = pd.DataFrame(pages)
df.shape

(1186, 7)

In [22]:
df = df.drop_duplicates()

In [23]:
df.head()

,Date,In reality,Link,PolitiFact source,Score,Source,Statement
0,"on Thursday, July 25th, 2019",Needs context,/punditfact/statements/2019/jul/25/roland-mart...,— PunditFact,Mostly True,Roland Martin,"Republicans ""are advancing federal judges who ..."
1,"on Tuesday, July 23rd, 2019",A joke misunderstood,/punditfact/statements/2019/jul/23/occupy-demo...,— PunditFact,False,Occupy Democrats,"""Fox ‘News’ just announced that they will not ..."
2,"on Monday, July 15th, 2019",It didn't go away,/punditfact/statements/2019/jul/15/rush-limbau...,— PunditFact,False,Rush Limbaugh,"""Somebody in the Obama regime decided to get r..."
3,"on Tuesday, July 2nd, 2019",They support a different approach,/punditfact/statements/2019/jul/02/jess-mcinto...,— PunditFact,Mostly False,Jess McIntosh,"""Fifty percent of Republicans are in favor of ..."
4,"on Monday, July 1st, 2019",Racial gap has persisted for years,/punditfact/statements/2019/jul/01/ta-nehisi-c...,— PunditFact,True,Ta-Nehisi Coates,"""The typical black family in this country has ..."


In [24]:
df.to_csv("punditfacts_data.csv")

# Preparing the data for graphing/analysis

In [26]:
df['Year'] = df.Date.str.extract(r'(\d{4})')

In [27]:
vc = df.Year.value_counts()

In [28]:
df2 = pd.DataFrame(df.groupby(by='Year').Score.value_counts())

In [30]:
df2 = df2.rename(columns={'Score':'Count'})

In [32]:
df2 = df2.reset_index()

In [33]:
df2 = df2.set_index('Year')

In [35]:
percents = df2.Count / vc

In [36]:
percents

2013    0.263158
2013    0.228070
2013    0.175439
2013    0.157895
2013    0.140351
2013    0.035088
2014    0.216718
2014    0.198142
2014    0.173375
2014    0.173375
2014    0.126935
2014    0.111455
2015    0.193069
2015    0.188119
2015    0.178218
2015    0.168317
2015    0.148515
2015    0.123762
2016    0.272727
2016    0.246753
2016    0.168831
2016    0.142857
2016    0.103896
2016    0.064935
2017    0.730216
2017    0.143885
2017    0.057554
2017    0.032374
2017    0.025180
2017    0.010791
2018    0.513043
2018    0.256522
2018    0.082609
2018    0.069565
2018    0.043478
2018    0.030435
2018    0.004348
2019    0.368421
2019    0.263158
2019    0.157895
2019    0.105263
2019    0.052632
2019    0.052632
dtype: float64

In [37]:
df2['Percent'] = percents

In [38]:
df2.Percent = df2.Percent*100

In [45]:
df2.groupby(by='Year').Count.sum()

Year
2013     57
2014    323
2015    202
2016     77
2017    278
2018    230
2019     19
Name: Count, dtype: int64

In [43]:
df2.to_csv("punditfacts_score_freq.csv")